In [8]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('MABe-mouse-behavior-detection/train.csv')
df

,lab_id,video_id,mouse1_strain,mouse1_color,mouse1_sex,mouse1_id,mouse1_age,mouse1_condition,mouse2_strain,mouse2_color,...,pix_per_cm_approx,video_width_pix,video_height_pix,arena_width_cm,arena_height_cm,arena_shape,arena_type,body_parts_tracked,behaviors_labeled,tracking_method
0,AdaptableSnail,44566106,CD-1 (ICR),white,male,10.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1228,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
1,AdaptableSnail,143861384,CD-1 (ICR),white,male,3.0,8-12 weeks,NaN,CD-1 (ICR),white,...,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
2,AdaptableSnail,209576908,CD-1 (ICR),white,male,7.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1266,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
3,AdaptableSnail,278643799,CD-1 (ICR),white,male,11.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1224,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
4,AdaptableSnail,351967631,CD-1 (ICR),white,male,14.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1204,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8785,UppityFerret,1610683375,C57Bl/6N,black,male,8.0,8 weeks,csds,CD1,white,...,13.0,664,572,38.0,38.0,circular,neutral,"[""body_center"", ""ear_left"", ""ear_right"", ""hip_...","[""mouse1,mouse2,reciprocalsniff"", ""mouse1,mous...",DeepLabCut
8786,UppityFerret,1725214092,C57Bl/6N,black,male,27.0,8 weeks,NaN,CD1,white,...,12.8,608,576,38.0,38.0,circular,neutral,"[""body_center"", ""ear_left"", ""ear_right"", ""hip_...","[""mouse1,mouse2,intromit"", ""mouse1,mouse2,moun...",DeepLabCut
8787,UppityFerret,1960237444,C57Bl/6N,black,male,14.0,8 weeks,csds,CD1,white,...,12.5,592,576,38.0,38.0,circular,neutral,"[""body_center"", ""ear_left"", ""ear_right"", ""hip_...","[""mouse1,mouse2,reciprocalsniff"", ""mouse1,mous...",DeepLabCut
8788,UppityFerret,2045808129,C57Bl/6N,black,male,12.0,8 weeks,csds,CD1,white,...,12.5,624,572,38.0,38.0,circular,neutral,"[""body_center"", ""ear_left"", ""ear_right"", ""hip_...","[""mouse1,mouse2,reciprocalsniff"", ""mouse1,mous...",DeepLabCut


In [3]:
df.columns

Index(['lab_id', 'video_id', 'mouse1_strain', 'mouse1_color', 'mouse1_sex',
       'mouse1_id', 'mouse1_age', 'mouse1_condition', 'mouse2_strain',
       'mouse2_color', 'mouse2_sex', 'mouse2_id', 'mouse2_age',
       'mouse2_condition', 'mouse3_strain', 'mouse3_color', 'mouse3_sex',
       'mouse3_id', 'mouse3_age', 'mouse3_condition', 'mouse4_strain',
       'mouse4_color', 'mouse4_sex', 'mouse4_id', 'mouse4_age',
       'mouse4_condition', 'frames_per_second', 'video_duration_sec',
       'pix_per_cm_approx', 'video_width_pix', 'video_height_pix',
       'arena_width_cm', 'arena_height_cm', 'arena_shape', 'arena_type',
       'body_parts_tracked', 'behaviors_labeled', 'tracking_method'],
      dtype='object')

In [ ]:
pip install --upgrade pyarrow


Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from pathlib import Path

# Pastas
base_folder = Path("MABe-mouse-behavior-detection")
tracking_folder = base_folder / "train_tracking"
annotation_folder = base_folder / "train_annotation"
df_meta = pd.read_csv(base_folder / "train.csv")

output_folder = base_folder / "processed_videos"
output_folder.mkdir(exist_ok=True)

def load_parquet(file_path: Path) -> pd.DataFrame:
    return pd.read_parquet(file_path)

def extract_video_id(file_path: Path) -> int:
    return int(file_path.stem)

# Itera sobre laboratórios
for lab_folder in tracking_folder.iterdir():
    if not lab_folder.is_dir():
        continue

    print(f"Processando laboratório {lab_folder.name}...")

    tracking_files = list(lab_folder.glob("*.parquet"))
    annotation_lab_folder = annotation_folder / lab_folder.name
    annotation_files = list(annotation_lab_folder.glob("*.parquet"))

    # Mapeia video_id -> annotation
    videoid_to_annotation = {extract_video_id(f): load_parquet(f) for f in annotation_files}

    for track_file in tracking_files:
        vid = extract_video_id(track_file)
        df_track = load_parquet(track_file)

        # Cria coluna de comportamento
        df_track['behavior'] = pd.NA
        df_ann = videoid_to_annotation.get(vid, pd.DataFrame(columns=['start_frame','stop_frame','action']))
        
        if not df_ann.empty:
            intervals = pd.IntervalIndex.from_arrays(df_ann['start_frame'], df_ann['stop_frame'], closed='both')
            for idx, interval in enumerate(intervals):
                mask = (df_track['video_frame'] >= interval.left) & (df_track['video_frame'] <= interval.right)
                df_track.loc[mask, 'behavior'] = df_ann['action'].iloc[idx]

        # Adiciona metadata
        meta_row = df_meta[df_meta['video_id'] == vid]
        if not meta_row.empty:
            for col in df_meta.columns:
                if col not in df_track.columns:
                    df_track[col] = meta_row.iloc[0][col]

        # Salva arquivo processado individual
        df_track.to_parquet(output_folder / f"{vid}_processed.parquet", index=False)
        print(f"Vídeo {vid} processado e salvo.")


Processando laboratório AdaptableSnail...
Vídeo 1212811043 processado e salvo.
Vídeo 1260392287 processado e salvo.
Vídeo 1351098077 processado e salvo.
Vídeo 1408652858 processado e salvo.
Vídeo 143861384 processado e salvo.
Vídeo 1596473327 processado e salvo.
Vídeo 1643942986 processado e salvo.
Vídeo 1717182687 processado e salvo.
Vídeo 2078515636 processado e salvo.
Vídeo 209576908 processado e salvo.
Vídeo 278643799 processado e salvo.
Vídeo 351967631 processado e salvo.
Vídeo 355542626 processado e salvo.
Vídeo 44566106 processado e salvo.
Vídeo 678426900 processado e salvo.
Vídeo 705948978 processado e salvo.
Vídeo 878123481 processado e salvo.
Processando laboratório BoisterousParrot...
Vídeo 1059582964 processado e salvo.
Vídeo 1184291605 processado e salvo.
Vídeo 1201849558 processado e salvo.
Vídeo 1459695188 processado e salvo.
Vídeo 1985626297 processado e salvo.
Vídeo 402963089 processado e salvo.
Vídeo 459610814 processado e salvo.
Vídeo 613246188 processado e salvo.
Pr

In [11]:
import pandas as pd
from pathlib import Path

# Pasta onde os arquivos processados foram salvos
processed_folder = Path("MABe-mouse-behavior-detection/processed_videos")

# Exemplo: carregar um vídeo pelo id
video_id = 1212811043
df_especifico = pd.read_parquet(processed_folder / f"{video_id}_processed.parquet")

df_especifico


,video_frame,mouse_id,bodypart,x,y,behavior,lab_id,video_id,mouse1_strain,mouse1_color,...,pix_per_cm_approx,video_width_pix,video_height_pix,arena_width_cm,arena_height_cm,arena_shape,arena_type,body_parts_tracked,behaviors_labeled,tracking_method
0,0,1,body_center,496.187012,376.475006,None,AdaptableSnail,1212811043,CD-1 (ICR),white,...,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
1,0,1,ear_left,494.059998,343.924011,None,AdaptableSnail,1212811043,CD-1 (ICR),white,...,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
2,0,1,ear_right,518.765015,367.362000,None,AdaptableSnail,1212811043,CD-1 (ICR),white,...,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
3,0,1,lateral_left,474.536987,370.563995,None,AdaptableSnail,1212811043,CD-1 (ICR),white,...,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
4,0,1,lateral_right,505.825012,394.937012,None,AdaptableSnail,1212811043,CD-1 (ICR),white,...,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2677001,89975,4,lateral_right,348.324005,584.469971,None,AdaptableSnail,1212811043,CD-1 (ICR),white,...,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
2677002,89975,4,nose,420.062988,585.309998,None,AdaptableSnail,1212811043,CD-1 (ICR),white,...,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
2677003,89975,4,tail_base,323.226013,561.976990,None,AdaptableSnail,1212811043,CD-1 (ICR),white,...,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
2677004,89975,4,tail_midpoint,287.417999,552.995972,None,AdaptableSnail,1212811043,CD-1 (ICR),white,...,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut


In [14]:
def add_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Incrementa o dataframe original com:
      - velocidade (vel)
      - ângulo (angle)
      - distância entre ratos (distancia)
    """

    # ============================
    # 1. Pegar apenas body_center
    # ============================
    centers = df[df["bodypart"] == "body_center"].copy()

    # ============================
    # 2. Calcular diferenças no tempo
    # ============================
    centers = centers.sort_values(by=["video_id", "mouse_id", "video_frame"])
    centers["dx"] = centers.groupby(["video_id", "mouse_id"])["x"].diff()
    centers["dy"] = centers.groupby(["video_id", "mouse_id"])["y"].diff()

    # ============================
    # 3. Calcular velocidade e ângulo
    # ============================
    centers["vel"] = np.sqrt(centers["dx"]**2 + centers["dy"]**2)
    centers["angle"] = np.arctan2(centers["dy"], centers["dx"])

    # ============================
    # 4. Calcular distância entre ratos
    # ============================
    dist_list = []
    for (vid, frame), group in centers.groupby(["video_id", "video_frame"]):
        if group["mouse_id"].nunique() >= 2:
            mice = group[["mouse_id", "x", "y"]].set_index("mouse_id")
            # distância apenas entre 1 e 2 (se houver)
            if {1, 2}.issubset(mice.index):
                x1, y1 = mice.loc[1, ["x", "y"]]
                x2, y2 = mice.loc[2, ["x", "y"]]
                dist = np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
                dist_list.append((vid, frame, dist))
    df_dist = pd.DataFrame(dist_list, columns=["video_id", "video_frame", "distancia"])

    # ============================
    # 5. Incrementar no original
    # ============================
    # primeiro junta vel/angle no df original
    df = df.merge(
        centers[["video_id", "video_frame", "mouse_id", "vel", "angle"]],
        on=["video_id", "video_frame", "mouse_id"],
        how="left"
    )

    # depois junta distância (mesma p/ todos os bodyparts no mesmo frame)
    df = df.merge(
        df_dist,
        on=["video_id", "video_frame"],
        how="left"
    )

    return df

In [17]:
df_novo = add_features(df_especifico)

In [18]:
df_novo.columns

Index(['video_frame', 'mouse_id', 'bodypart', 'x', 'y', 'behavior', 'lab_id',
       'video_id', 'mouse1_strain', 'mouse1_color', 'mouse1_sex', 'mouse1_id',
       'mouse1_age', 'mouse1_condition', 'mouse2_strain', 'mouse2_color',
       'mouse2_sex', 'mouse2_id', 'mouse2_age', 'mouse2_condition',
       'mouse3_strain', 'mouse3_color', 'mouse3_sex', 'mouse3_id',
       'mouse3_age', 'mouse3_condition', 'mouse4_strain', 'mouse4_color',
       'mouse4_sex', 'mouse4_id', 'mouse4_age', 'mouse4_condition',
       'frames_per_second', 'video_duration_sec', 'pix_per_cm_approx',
       'video_width_pix', 'video_height_pix', 'arena_width_cm',
       'arena_height_cm', 'arena_shape', 'arena_type', 'body_parts_tracked',
       'behaviors_labeled', 'tracking_method', 'vel', 'angle', 'distancia'],
      dtype='object')